In [3]:
import numpy as np
# 该模块为自定义模块，封装了构建决策树的基本方法
from cart import *
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# 树的棵数
n_estimators = 10
# 列抽样最大特征数
max_features = 15
# 生成模拟二分类数据集
X, y = make_classification(n_samples=1000, n_features=20, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(700, 20) (700,) (300, 20) (300,)


In [4]:
# 合并训练数据和标签
X_y = np.concatenate([X, y.reshape(-1,1)], axis=1)
np.random.shuffle(X_y)
m = X_y.shape[0]
sampling_subsets = []

for _ in range(n_estimators):
    idx = np.random.choice(m, m, replace=True)
    bootstrap_Xy = X_y[idx, :]
    bootstrap_X = bootstrap_Xy[:, :-1]
    bootstrap_y = bootstrap_Xy[:, -1]
    sampling_subsets.append([bootstrap_X, bootstrap_y])

In [8]:
sampling_subsets[0][0].shape

1000

In [10]:
# 自助抽样选择训练数据子集
def bootstrap_sampling(X, y):
    X_y = np.concatenate([X, y.reshape(-1,1)], axis=1)
    np.random.shuffle(X_y)
    n_samples = X.shape[0]
    sampling_subsets = [] #抽样子集

    for _ in range(n_estimators): #树的棵树
        # 第一个随机性，行抽样
        idx1 = np.random.choice(n_samples, n_samples, replace=True) #有放回随机抽样
        bootstrap_Xy = X_y[idx1, :]
        bootstrap_X = bootstrap_Xy[:, :-1]
        bootstrap_y = bootstrap_Xy[:, -1]
        sampling_subsets.append([bootstrap_X, bootstrap_y])
    return sampling_subsets

In [12]:
sampling_subsets = bootstrap_sampling(X_train, y_train)
sub_X, sub_y = sampling_subsets[0]
print(sub_X.shape, sub_y.shape)
type(sampling_subsets)

(700, 20) (700,)


list

In [15]:
trees = []
# 基于决策树构建森林
for _ in range(n_estimators):
    tree = ClassificationTree(min_samples_split=2, min_gini_impurity=999,
                              max_depth=3)
    trees.append(tree)

trees[0]

In [16]:
# 随机森林训练
def fit(X, y):
    # 对森林中每棵树训练一个双随机抽样子集
    n_features = X.shape[1]
    sub_sets = bootstrap_sampling(X, y)
    for i in range(n_estimators):
        sub_X, sub_y = sub_sets[i]
        # 第二个随机性，列抽样
        idx2 = np.random.choice(n_features, max_features, replace=True)
        sub_X = sub_X[:, idx2]
        trees[i].fit(sub_X, sub_y)
        trees[i].feature_indices = idx2
        print('The {}th tree is trained done...'.format(i+1))

fit(X_train, y_train)

/Users/heshuaichen/numpy/机器学习—公式推导与代码实现/10—16Ensemble_Learning_监督集成/utils.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([X_left, X_right])


The 1th tree is trained done...
The 2th tree is trained done...
The 3th tree is trained done...
The 4th tree is trained done...
The 5th tree is trained done...
The 6th tree is trained done...
The 7th tree is trained done...
The 8th tree is trained done...
The 9th tree is trained done...
The 10th tree is trained done...


In [29]:
y_preds = []
for i in range(n_estimators):
    idx = trees[i].feature_indices
    sub_X = X_test[:, idx]
    y_pred = trees[i].predict(sub_X)
    y_preds.append(y_pred)
    
len(y_preds[0])
type(y_preds)
np.array(y_preds).shape

(10, 300)

In [54]:
y_preds = np.array(y_preds).T
y_preds
a = []
for i in y_preds:
    print(i.astype('int'))
    a.append(i.astype('int'))
print(len(a))
print(a[1])


[0 1 0 0 1 1 1 1 1 0]
[0 0 0 0 0 1 1 1 0 0]
[0 0 0 0 0 1 1 0 0 0]
[1 0 0 0 0 1 1 0 0 1]
[0 0 0 0 0 1 0 1 0 0]
[0 1 1 1 1 1 1 1 1 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[1 1 1 1 1 1 0 1 1 1]
[0 0 0 0 0 1 1 0 0 0]
[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 1 1 0 0 0]
[1 1 1 1 1 0 1 1 1 1]
[0 0 0 0 0 1 1 0 0 0]
[0 1 1 1 1 1 1 1 1 0]
[0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 1 1 0 1]
[0 0 1 0 1 0 0 1 1 0]
[0 1 1 1 1 1 0 1 1 0]
[1 0 1 0 0 1 1 0 0 0]
[0 1 1 1 1 0 1 1 1 0]
[1 1 0 0 0 1 0 0 1 0]
[0 0 1 0 0 1 0 0 1 0]
[0 1 1 1 1 1 0 1 1 0]
[0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 1 0 1]
[0 1 0 0 0 1 1 1 1 0]
[0 0 0 0 0 1 1 0 0 0]
[1 0 0 0 1 1 1 0 1 0]
[0 0 0 0 0 1 1 0 0 1]
[0 0 0 0 0 1 1 0 0 0]
[1 0 0 0 0 1 0 0 0 1]
[0 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 1 0 0 0]
[0 1 1 1 1 1 0 1 1 0]
[1 1 1 1 1 1 1 0 1 0]
[1 1 1 1 1 1 1 1 1 0]
[0 1 0 0 0 1 1 0 0 0]
[0 0 0 0 0 1 1 0 0 0]
[0 0 0 1 0 0 0 1 1 1]
[0 0 0 0 0 1 1 0 0 1]
[1 0 0 0 0 1 1 0 0 1]
[0 1 1 1 1 1 1 1 1 0]
[1 0 0 0 0 1 1 0 0 1]
[0 1 1 1 1 1 0 1 1 0]
[0 0 0 0 0

In [55]:
np.bincount(a[1])

array([7, 3])

In [56]:
np.bincount(a[1]).argmax() #达到了取多数类为预测累的效果

0

In [18]:
y_preds = np.array(y_preds).T
print(y_preds.shape)
y_pred = []
for y_p in y_preds:
    y_pred.append(np.bincount(y_p.astype('int')).argmax()) # .argmax()返回了最大值索引 np.bincount频数统计
    #意义为返回频数最大的索引
print(y_pred[:10])

(300, 10)
[1, 0, 0, 0, 0, 1, 0, 0, 1, 0]


In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.78


In [65]:
class RandomForest():
    def __init__(self, n_estimators=100, min_samples_split=2, min_gini_impurity=0,
                 max_depth=float("inf"), max_features=None):
        # 树的棵树
        self.n_estimators = n_estimators
        # 树最小分裂样本数
        self.min_samples_split = min_samples_split
        # 最小增益
        self.min_gini_impurity = min_gini_impurity
        # 树最大深度
        self.max_depth = max_depth
        # 所使用最大特征数
        self.max_features = max_features

        self.trees = []
        # 基于决策树构建森林
        for _ in range(self.n_estimators):
            tree = ClassificationTree(min_samples_split=self.min_samples_split, min_gini_impurity=self.min_gini_impurity,
                                      max_depth=self.max_depth)
            self.trees.append(tree)
            
    # 自助抽样
    def bootstrap_sampling(self, X, y):
        X_y = np.concatenate([X, y.reshape(-1,1)], axis=1)
        np.random.shuffle(X_y)
        n_samples = X.shape[0]
        sampling_subsets = []

        for _ in range(self.n_estimators):
            # 第一个随机性，行抽样
            idx1 = np.random.choice(n_samples, n_samples, replace=True)
            bootstrap_Xy = X_y[idx1, :]
            bootstrap_X = bootstrap_Xy[:, :-1]
            bootstrap_y = bootstrap_Xy[:, -1]
            sampling_subsets.append([bootstrap_X, bootstrap_y])
        return sampling_subsets
            
    # 随机森林训练
    def fit(self, X, y):
        # 对森林中每棵树训练一个双随机抽样子集
        sub_sets = self.bootstrap_sampling(X, y)
        n_features = X.shape[1]
        # 设置max_feature
        if self.max_features == None:
            self.max_features = int(np.sqrt(n_features))
        
        for i in range(self.n_estimators):
            # 第二个随机性，列抽样
            sub_X, sub_y = sub_sets[i]
            idx2 = np.random.choice(n_features, self.max_features, replace=True)
            sub_X = sub_X[:, idx2]
            self.trees[i].fit(sub_X, sub_y)
            # 保存每次列抽样的列索引，方便预测时每棵树调用
            self.trees[i].feature_indices = idx2
            print('The {}th tree is trained done...'.format(i+1))
    
    # 随机森林预测
    def predict(self, X):
        y_preds = []
        for i in range(self.n_estimators):
            idx = self.trees[i].feature_indices
            sub_X = X[:, idx]
            y_pred = self.trees[i].predict(sub_X)
            y_preds.append(y_pred)
            
        y_preds = np.array(y_preds).T
        res = []
        for j in y_preds:
            res.append(np.bincount(j.astype('int')).argmax())
        return res

In [66]:
rf = RandomForest(n_estimators=10, max_features=15)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))

/Users/heshuaichen/numpy/机器学习—公式推导与代码实现/10—16Ensemble_Learning_监督集成/utils.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([X_left, X_right])


The 1th tree is trained done...
The 2th tree is trained done...
The 3th tree is trained done...
The 4th tree is trained done...
The 5th tree is trained done...
The 6th tree is trained done...
The 7th tree is trained done...
The 8th tree is trained done...
The 9th tree is trained done...
The 10th tree is trained done...
0.4866666666666667


In [67]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7733333333333333
